# Importing the necessary packages

In [23]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

# Creating the columns of the dataset 

By this part, we're going to create the columns by the format of lists that we're going to use in our dataframe. Right now they're empty, but they'll be filled soon.

In [2]:
authors = []
dates = []
statements = []
sources = []
targets = []

# Creating a function to turn it easier to web scrap

In [6]:
def scrape_website(page_number):
    page_num = str(page_number)
    URL = 'https://www.politifact.com/factchecks/list/?page='+page_num #Setting the URL of the site we're scrapping
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.text, 'html.parser')
    statement_footer = soup.find_all ('footer' , attrs={'class' : 'm-statement__footer'})
    statement_quote = soup.find_all ('div' , attrs={'class' : 'm-statement__quote'})
    statement_meta = soup.find_all ('div' , attrs={'class' : 'm-statement__meta'})
    target = soup.find_all ('div' , attrs={'class' : 'm-statement__meter'})

    for i in statement_footer:
        link1 = i.text.strip()
        name_and_date= link1.split()
        first_name = name_and_date[1]
        last_name = name_and_date[2]
        full_name = first_name + ' ' + last_name
        month = name_and_date[4]
        day = name_and_date[5]
        year = name_and_date[6]
        date = month+' '+day+' '+year
        dates.append(date)
        authors.append(full_name)   

    for i in statement_quote:
        link2 = i.find_all('a')
        statement_text = link2[0].text.strip()
        statements.append(statement_text)

    for i in statement_meta:
        link3 = i.find_all('a')
        source_text = link3[0].text.strip()
        sources.append(source_text)

    for i in target:
        link4 = i.find('div' , attrs={'class' : 'c-image'}).find('img').get('alt')
        targets.append(link4)

In [7]:
n = 50
for i in range(1, n):
    scrape_website(i) #This function was created specifically to scrap this site, so to scrap data from other sites it might not work

In [8]:
data = pd.DataFrame(columns = ['author','statement','source','date','target'])
data['author']= authors
data['statement']= statements
data['source']= sources
data['date']= dates
data['target']= targets

In [9]:
data.head()

,author,statement,source,date,target
0,Jeff Cercone,Washington’s State Board of Health will discus...,Facebook posts,"January 11, 2022",barely-true
1,Ciara O'Rourke,Says Supreme Court Justice Sonia Sotomayor was...,Facebook posts,"January 11, 2022",false
2,Madison Czopek,Walgreens refrigerators are scanning shoppers’...,Facebook posts,"January 11, 2022",pants-fire
3,Nina Baker,"Gov. Kim Reynolds, touting $210 million for Io...",Iowa Senate Democrats,"January 11, 2022",true
4,Samantha Putterman,“New Zealand okays euthanasia for COVID patien...,Bloggers,"January 11, 2022",barely-true


In [10]:
data.shape

(2340, 5)

In [19]:
true = ['mostly-true', 'true']

def  getBinaryNumTarget(text):
    if text in true  :
            return 0
    else:
        return 1
        

In [20]:
data['BinaryNumTarget']= data['target'].apply(getBinaryNumTarget)


In [21]:
data.head(20)

,author,statement,source,date,target,BinaryNumTarget
0,Jeff Cercone,Washington’s State Board of Health will discus...,Facebook posts,"January 11, 2022",barely-true,1
1,Ciara O'Rourke,Says Supreme Court Justice Sonia Sotomayor was...,Facebook posts,"January 11, 2022",false,1
2,Madison Czopek,Walgreens refrigerators are scanning shoppers’...,Facebook posts,"January 11, 2022",pants-fire,1
3,Nina Baker,"Gov. Kim Reynolds, touting $210 million for Io...",Iowa Senate Democrats,"January 11, 2022",true,0
4,Samantha Putterman,“New Zealand okays euthanasia for COVID patien...,Bloggers,"January 11, 2022",barely-true,1
5,Amy Sherman,Democrats “are trying to ban voter ID.”,Donald Trump,"January 11, 2022",barely-true,1
6,Jeff Cercone,"""Clarence Thomas’s wife was one of the organiz...",Facebook posts,"January 11, 2022",false,1
7,Vivian Lam,"""Biden's CDC Director ADMITS Her Own Employees...",Facebook posts,"January 11, 2022",false,1
8,Ciara O'Rourke,“Barack Obama’s real father is Communist Party...,Viral image,"January 10, 2022",pants-fire,1
9,Madison Czopek,“A tragic end today” for CNN’s Sanjay Gupta,Facebook posts,"January 10, 2022",false,1


In [22]:
data.to_csv('fact_checker.csv')